In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("IR Dec25_Cluster_Analysis.xlsx")

In [3]:
df.head()

,Timestamp (UTC),hl,oc,hl-oc,hl/vol,Volume,5DayRollingStdDev,Cluster
0,2024-02-08 06:08:00,0.04,0.01,0.03,0.000082,489,0.013038,0
1,2024-02-08 14:08:00,0.02,0.02,0.00,0.000127,157,0.019235,0
2,2024-02-08 21:28:00,0.04,0.02,0.02,0.000037,1077,0.022804,3
3,2024-02-09 05:28:00,0.00,0.00,0.00,0.000000,5,0.016733,0
4,2024-02-09 06:08:00,0.04,0.03,0.01,0.000028,1436,0.022804,2


## 1. Get **Previous State**
**using .shift(1) on "Cluster" column**

In [5]:
df['previous_state'] = df['Cluster'].shift(1)

In [6]:
df.head()

,Timestamp (UTC),hl,oc,hl-oc,hl/vol,Volume,5DayRollingStdDev,Cluster,previous_state
0,2024-02-08 06:08:00,0.04,0.01,0.03,0.000082,489,0.013038,0,NaN
1,2024-02-08 14:08:00,0.02,0.02,0.00,0.000127,157,0.019235,0,0.0
2,2024-02-08 21:28:00,0.04,0.02,0.02,0.000037,1077,0.022804,3,0.0
3,2024-02-09 05:28:00,0.00,0.00,0.00,0.000000,5,0.016733,0,3.0
4,2024-02-09 06:08:00,0.04,0.03,0.01,0.000028,1436,0.022804,2,0.0


In [7]:
# drop first row
df1=df.dropna()
df1 = df1.copy()  # Make an explicit copy of the DataFrame

In [8]:
# converting 'previous_state' from float to int
df1['previous_state'] = df1['previous_state'].astype(int)

In [9]:
#df1.head()

## 2. Creating **Transition States**

In [11]:
df1['c_states'] = df1['previous_state'].astype(str) + df1['Cluster'].astype(str)

In [12]:
df1.head(10)

,Timestamp (UTC),hl,oc,hl-oc,hl/vol,Volume,5DayRollingStdDev,Cluster,previous_state,c_states
1,2024-02-08 14:08:00,0.02,0.02,0.00,0.000127,157,0.019235,0,0,00
2,2024-02-08 21:28:00,0.04,0.02,0.02,0.000037,1077,0.022804,3,0,03
3,2024-02-09 05:28:00,0.00,0.00,0.00,0.000000,5,0.016733,0,3,30
4,2024-02-09 06:08:00,0.04,0.03,0.01,0.000028,1436,0.022804,2,0,02
5,2024-02-09 14:08:00,0.05,0.02,0.03,0.000099,505,0.030332,0,2,20
6,2024-02-11 21:28:00,0.03,0.00,0.03,0.000046,655,0.031145,3,0,03
7,2024-02-12 06:08:00,0.01,0.00,0.01,0.000095,105,0.031145,0,3,30
8,2024-02-12 14:08:00,0.01,0.01,0.00,0.000070,143,0.015811,0,0,00
9,2024-02-12 21:28:00,0.03,0.00,0.03,0.000019,1547,0.011402,2,0,02
10,2024-02-13 05:28:00,0.01,0.00,0.01,0.000208,48,0.010954,0,2,20


## 3. Grouping by Transition state and Counting occurrences

In [14]:
states = df1.groupby('c_states')['c_states'].value_counts().fillna(0).reset_index()

In [15]:
states

,c_states,count
0,00,20
1,01,1
2,02,21
3,03,24
4,10,4
5,20,18
6,22,4
7,23,8
8,30,24
9,31,3


## 4. Separate the transition state
**'previous' and 'present' state to 'start' and 'end' state**

In [17]:
# Extract start and end states from `c_states`
states["start_state"] = states["c_states"].str[0].astype(int)
states["end_state"] = states["c_states"].str[1].astype(int)

In [18]:
states

,c_states,count,start_state,end_state
0,00,20,0,0
1,01,1,0,1
2,02,21,0,2
3,03,24,0,3
4,10,4,1,0
5,20,18,2,0
6,22,4,2,2
7,23,8,2,3
8,30,24,3,0
9,31,3,3,1


## 5. Create **Markov transition matrix**

In [20]:
# Create the Markov transition matrix
transition_matrix = states.pivot(index="start_state", columns="end_state", values="count").fillna(0)

In [21]:
transition_matrix

end_state,0,1,2,3
start_state,,,,
0,20.0,1.0,21.0,24.0
1,4.0,0.0,0.0,0.0
2,18.0,0.0,4.0,8.0
3,24.0,3.0,5.0,10.0


## 6. Convert to row-wise percentages

In [23]:
transition_matrix_perc = transition_matrix.div(transition_matrix.sum(axis=1), axis=0) * 100

In [24]:
transition_matrix_perc.round(2)

end_state,0,1,2,3
start_state,,,,
0,30.30,1.52,31.82,36.36
1,100.00,0.00,0.00,0.00
2,60.00,0.00,13.33,26.67
3,57.14,7.14,11.90,23.81


**Applications:**

Prediction: Predict the next state of the market based on the current state (i.e., forecasting the market's behavior).

Optimization: Use the Markov Model for portfolio optimization, where states represent different market conditions, and transitions model how the market shifts.

Risk Assessment: Assess the risk of being in a certain state at a future time.